In [ ]:
### 레퍼런스
#- API Reference: https://platform.openai.com/docs/api-reference/audio/createTranscription
from openai import OpenAI
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# OpenAI 클라이언트 전역 변수 선언
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def transcribe(audio_filepath, file_name, topics):
    #오디오 파일을 텍스트로 변환
    full_path = os.path.join(audio_filepath, file_name)
    with open(full_path, "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            prompt=topics,
        )
    return transcription.text

def generate_prompt_from_filename(filename):
    #파일명에서 프롬프트를 생성
    base_name = os.path.basename(filename)
    name_without_extension = os.path.splitext(base_name)[0]
    return name_without_extension.replace('_', '와 ')

def generate_prompt_from_user_input():
    #사용자 입력으로부터 프롬프트를 생성
    user_input = input("회의 주제: ")
    return f"너는 회의록을 정리하는 팀원이야. 회의록의 핵심 내용을 정리해줘. 회의 주제는 {user_input}이야."



def generate_corrected_transcript(transcribed_text, prompt):
    #텍스트를 요약하여 정리된 회의록을 생성
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0.5,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": transcribed_text}
        ],
    )
    return completion.choices[0].message.content

def main():
    audio_filepath = "../data"
    file_name = "company_meeting_stock_lunch.m4a"
    prompt = generate_prompt_from_filename(file_name)
    system_prompt = generate_prompt_from_user_input()

    transcribed_text = transcribe(audio_filepath, file_name, prompt)
    corrected_text = generate_corrected_transcript(transcribed_text, system_prompt)

    print(corrected_text)

if __name__ == "__main__":
    main()